# Preparar el Dataset para entrenamiento y evaluación

# Amazon Customer Reviews Dataset

https://s3.amazonaws.com/amazon-reviews-pds/readme.html

## Esquema del Dataset

- `marketplace`: Código de país de 2 letras (en este caso todo "US").
- `customer_id`: Identificador aleatorio que puede utilizarse para agregar las reseñas escritas por un solo autor.
- `review_id`: Un ID único para la review.
- `product_id`: El [Amazon Standard Identification Number (ASIN)]; `http://www.amazon.com/dp/<ASIN>` enlaces a la página de detalles del producto.
- `product_parent`: El padre de ese ASIN.  Múltiples ASINs (variaciones de color o formato de un mismo producto) pueden emparentarse en una única matriz.
- `product_title`: Descripción del título del producto.
- `product_category`: Amplia categoría de productos que puede utilizarse para agrupar las reseñas (en este caso, vídeos digitales).
- `star_rating`: La valoración de la reseña (de 1 a 5 estrellas).
- `helpful_votes`: Número de votos útiles para la revisión.
- `total_votes`: Número de votos totales que ha recibido la reseña.
- `vine`: ¿La revisión fue escrita como parte del programa [Vine](https://www.amazon.com/gp/vine/help)?
- `verified_purchase`: ¿Fue la revisión de una compra verificada?
- `review_headline`: El propio título de la reseña.
- `review_body`: El texto de la revisión.
- `review_date`: La fecha en que se escribió la reseña.

## Check dependencies setup

In [ ]:
%store -r setup_dependencies_passed

try:
    setup_dependencies_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOK ")
    print("You did not install the required libraries.   ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

if not setup_dependencies_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOK ")
    print("You did not install the required libraries.   ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK] Dependencies correctly set up")

## Setup notebook

In [ ]:
import boto3
import sagemaker
import pandas as pd
import csv
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

%matplotlib inline
%config InlineBackend.figure_format='retina'

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

## Descarga

Empecemos por recuperar un subconjunto del Dataset de las opiniones de los clientes de Amazon.

In [ ]:
!mkdir -p ./tmp
!aws s3 cp 's3://amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Software_v1_00.tsv.gz' ./tmp/

temp_folder = "tmp"
dataset = "amazon_reviews_us_Digital_Software_v1_00.tsv.gz"
dataset_csv = "amazon_reviews_us_Digital_Software_v1_00_comprehend.csv"

In [ ]:
df = pd.read_csv(
    f"{temp_folder}/{dataset}",
    delimiter="\t",
    quoting=csv.QUOTE_NONE,
    compression="gzip",
)

In [ ]:
print(f"Shape of dataframe before splitting {df.shape}")

In [ ]:
df.head(5)

In [ ]:
df[["star_rating", "review_id"]].groupby("star_rating").count().plot(kind="bar", title="Breakdown by Star Rating")

plt.xlabel("Star Rating")
plt.ylabel("Review Count")

# Balancear el Dataset

Para balancear el dataset, primero agrupamos por star_rating, y después seleccionamos un número de muestras:

- o bien valor más bajo de star_rating (_6890)
- o bien se le da un valor fijo para obtener 10_000 por ejemplo

Para este ejercicio vamos a escoger un número limitado de muestras del dataset para agilizar los procesos y abaratar los costes

Observamos el dataset agrupado por `star_rating`

In [ ]:
df_grouped_by = df.groupby(["star_rating"]) 
df_grouped_by.sum(numeric_only=True)

Obtenemos el valor mínimo

In [ ]:
df_grouped_by.size().min()

Reducimos cada agrupación al número de muestras apropiado

In [ ]:
# df_balanced = df_grouped_by.apply(lambda x: x.sample(df_grouped_by.size().min()).reset_index(drop=True))
df_balanced = df_grouped_by.apply(lambda x: x.sample(200).reset_index(drop=True))

df_balanced = df_balanced.reset_index(drop=True)

print(f"Shape of balanced dataframe {df_balanced.shape}")

Observamos de nuevo la dispersión

In [ ]:
df_balanced[["star_rating", "review_id"]].groupby("star_rating").count().plot(
    kind="bar", title="Breakdown by Star Rating"
)
plt.xlabel("Star Rating")
plt.ylabel("Review Count")

In [ ]:
df_balanced.head(5)

## Separamos los datos en sets train, validation y test

In [ ]:
# Separamos todos los datos en 90% entranmiento y 10% retenidos para pruebas
df_train, df_holdout = train_test_split(df_balanced, test_size=0.10, stratify=df_balanced["star_rating"])

# Separamos los datos retenidos para pruebas en 50% validación y 50% test
df_validation, df_test = train_test_split(df_holdout, test_size=0.50, stratify=df_holdout["star_rating"])

In [ ]:
# Pie chart, donde los cortes se ordenarán y trazarán en sentido contrario a las agujas del reloj:
labels = ["Train", "Validation", "Test"]
sizes = [len(df_train.index), len(df_validation.index), len(df_test.index)]
explode = (0.1, 0, 0)

fig1, ax1 = plt.subplots()

ax1.pie(sizes, explode=explode, labels=labels, autopct="%1.1f%%", startangle=90)

# Una relación de aspecto igual garantiza que el pastel se dibuje como un círculo.
ax1.axis("equal")

plt.show()

## Mostramos el set del 90% de datos de entrenamiento

In [ ]:
df_train.shape

In [ ]:
df_train[["star_rating", "review_id"]].groupby("star_rating").count().plot(
    kind="bar", title="90% Train Breakdown by Star Rating"
)

## Mostramos el set del 90% de datos de validación

In [ ]:
df_validation.shape

In [ ]:
df_validation[["star_rating", "review_id"]].groupby("star_rating").count().plot(
    kind="bar", title="5% Validation Breakdown by Star Rating"
)

## Mostramos el set del 90% de datos de test

In [ ]:
df_test.shape

In [ ]:
df_test[["star_rating", "review_id"]].groupby("star_rating").count().plot(
    kind="bar", title="5% Test Breakdown by Star Rating"
)

## Seleccionamos las features `star_rating` y `review_body` para el entrenamiento.

In [ ]:
df_train = df_train[["star_rating", "review_body"]]
df_train.shape

In [ ]:
df_train.head(5)

## Escribimos en un CSV sin cabeceras para Comprehend

In [ ]:
comprehend_train_path = f"{temp_folder}/{dataset_csv}"
df_train.to_csv(comprehend_train_path, index=False, header=False)

## Subimos los datos de entramiento a S3 para ser consumidos por Comprehend

In [ ]:
train_s3_prefix = "data"
comprehend_train_s3_uri = sess.upload_data(path=comprehend_train_path, key_prefix=train_s3_prefix)
comprehend_train_s3_uri

In [ ]:
!aws s3 ls $comprehend_train_s3_uri

## Almacenamos la URI con nuestros datos de entrenamiento en el servidor de Jupyter para usarlos en el siguiente notebook

In [ ]:
%store comprehend_train_s3_uri

In [ ]:
%store

# Release Resources

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>